In [1]:
import torch

from rlaopt.solvers import PCGConfig, SAPConfig, SAPAccelConfig
from rlaopt.preconditioners import NystromConfig

from scalable_gp_inference.kernel_linsys import KernelLinSys

In [2]:
torch.set_default_dtype(torch.float64)
torch.manual_seed(0)

In [3]:
n = 10000
d = 3
k = 10
reg = 1e-6 * n
kernel_type = "rbf"
kernel_lengthscale = 1.0
residual_tracking_idx = None
distributed = False
devices = None
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

X = torch.randn(n, d, device=device)
B = torch.randn(n, k, device=device)

In [4]:
kernel_linsys = KernelLinSys(X, B, reg, kernel_type, kernel_lengthscale, 
                            residual_tracking_idx, distributed, devices)

In [5]:
nystrom_config = NystromConfig(rank=100, rho=reg, damping_mode="adaptive")
# accel_config = SAPAccelConfig(mu=reg, nu=10.0)
# solver_config = SAPConfig(
#     precond_config=nystrom_config,
#     max_iters=1000,
#     atol=1e-6,
#     rtol=1e-6,
#     blk_sz=n // 10,
#     accel_config=accel_config,
#     device=device,
# )
solver_config = PCGConfig(
    precond_config=nystrom_config,
    max_iters=1000,
    atol=1e-6,
    rtol=1e-6,
    device=device,
)

In [6]:
kernel_linsys.solve(
    solver_config=solver_config,
    W_init=torch.zeros_like(B),
    log_in_wandb=True,
    wandb_init_kwargs={"project": "test_krr_linsys_class"},
)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: pratikrathore8 (sketchy-opts) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


[pyKeOps] Warning : at least one of the input tensors is not contiguous. Consider using contiguous data arrays to avoid unnecessary copies.


wandb: ERROR The nbformat package was not found. It is required to save notebook history.


cum_time,▁▂▃▄▅▆▇█
iter_time,█▃▁▁▁▁▁▁
cum_time,2.30414
iter_time,0.24615


(tensor([[  38.5156,   28.1211,   93.2924,  ...,   12.6360, -108.1160,
            87.4385],
         [  30.0534,   35.5796,  187.8935,  ..., -154.0083,  -73.0118,
           187.8499],
         [-180.4548,   97.5761, -223.6069,  ...,  -43.8714,   43.5497,
          -103.3113],
         ...,
         [ -89.7680,  -98.5448,  134.6107,  ...,   48.5665,  -34.9134,
           104.2638],
         [  99.1478,   87.0454,   -7.8039,  ...,  182.6885, -172.6538,
          -177.6213],
         [ -72.3226,   96.8327,   -6.3978,  ..., -222.5742,   36.8841,
            56.9417]], device='cuda:0'),
 {0: {'iter_time': 0.5092678070068359,
   'cum_time': 0.5092678070068359,
   'metrics': {'internal_metrics': {'abs_res': tensor([100.1735,  99.6921, 100.2273, 100.2879, 101.1664, 100.2388, 100.5417,
              99.1263,  98.5769, 100.1025], device='cuda:0'),
     'rel_res': tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], device='cuda:0')}}},
  10: {'iter_time': 0.3170309066772461,
   'cum_time': 0.82629